In [4]:
from datetime import date,timedelta,datetime
import calendar as ca
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import lightgbm as lgb

In [5]:
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

In [6]:
print('Loading Data')
df_train = pd.read_csv(
    'E:\DQS\kaggle\input/train_2017.csv', usecols=[1, 2, 3, 4, 5],
    dtype={'onpromotion': bool},
    converters={'unit_sales': lambda u: np.log1p(
        float(u)) if float(u) > 0 else 0},
    parse_dates=["date"]
)
item_nbr_u = df_train[df_train.date>pd.datetime(2017,8,10)].item_nbr.unique()

df_test = pd.read_csv(
    "E:\DQS\kaggle\input/test.csv", usecols=[0, 1, 2, 3, 4],
    dtype={'onpromotion': bool},
    parse_dates=["date"]  # , date_parser=parser
).set_index(
    ['store_nbr', 'item_nbr', 'date']
)

items = pd.read_csv(
    "E:\DQS\kaggle\input/items.csv",
).set_index("item_nbr")

df_2017 = df_train.loc[df_train.date>=pd.datetime(2017,1,1)]
del df_train

Loading Data


In [7]:
promo_2017_train_NaN = df_2017.set_index(
    ["store_nbr", "item_nbr", "date"])[["onpromotion"]].unstack(
        level=-1)
promo_2017_train = promo_2017_train_NaN.fillna(False)
promo_2017_train_NaN.columns = promo_2017_train_NaN.columns.get_level_values(1)
promo_2017_train.columns = promo_2017_train.columns.get_level_values(1)
promo_2017_test = df_test[["onpromotion"]].unstack(level=-1).fillna(False)
promo_2017_test.columns = promo_2017_test.columns.get_level_values(1)
promo_2017_test = promo_2017_test.reindex(promo_2017_train.index).fillna(False)
promo_2017 = pd.concat([promo_2017_train, promo_2017_test], axis=1)

print("the shape of promo_2017 : ",promo_2017.shape)
del promo_2017_test, promo_2017_train

df_2017_NaN = df_2017.set_index(
    ["store_nbr", "item_nbr", "date"])[["unit_sales"]].unstack(
        level=-1)
df_2017_NaN.columns = df_2017_NaN.columns.get_level_values(1)
df_2017 = df_2017_NaN.fillna(0)
print("the shape of df_2017 : ",df_2017.shape)
print("the shape of df_2017.reset_index():",df_2017.reset_index().shape)  

items = items.reindex(df_2017.index.get_level_values(1))

the shape of promo_2017 :  (167515, 243)
the shape of df_2017 :  (167515, 227)
the shape of df_2017.reset_index(): (167515, 229)


In [8]:
sales = df_2017.stack().to_frame('unit_sales')
sales.index.set_names(['store_nbr','item_nbr','date'],inplace=True)
sales = sales.reset_index()
print("the shape of sales : ",sales.shape)
item = pd.read_csv("E:\DQS\kaggle\input/items.csv")
store = pd.read_csv("E:\DQS\kaggle\input/stores.csv")

sales = pd.merge(sales,store,how='left',on=['store_nbr'])
sales = pd.merge(sales,item,how='left',on=['item_nbr'])

sales['country'] = 'Ecuador'

column_names = sales.drop_duplicates([
    'store_nbr','item_nbr','city','family','class','country','state','type','cluster','perishable'])
print("the shape of column_names : ",column_names.shape)
del item, store, trans

the shape of sales :  (38025905, 4)
the shape of column_names :  (167515, 13)


In [9]:
def A_B(str_A,str_B,content='unit_sales'): ##
    print("=============================================")
    A_B_tmp = sales[['date',content,str_A,str_B]].groupby([str_A,str_B,'date']).sum().unstack(level=-1)
    A_B_tmp.columns = A_B_tmp.columns.get_level_values(1)
    print("the shape of {}_{} is {}: ".format(str_A, str_B, A_B_tmp.shape))
    return A_B_tmp
def country_B(str_B,content='unit_sales'): ##
    print("=============================================")
    country_B_tmp = sales[['date',content,str_B]].groupby([str_B,'date']).sum().unstack(level=-1)
    country_B_tmp.columns = country_B_tmp.columns.get_level_values(1)
    print("the shape of country_{} is {}: ".format(str_B, country_B_tmp.shape))
    return country_B_tmp


In [10]:
print("Preparing unit_sales..........................")
#0 sales
family_item = A_B('family','item_nbr')
country_store = country_B('store_nbr')
country_perishable = country_B('perishable')
country_class = country_B('class')
cluster_item = A_B('cluster','item_nbr')
state_item = A_B('state','item_nbr')
type_item = A_B('type','item_nbr')
city_item = A_B('city','item_nbr')

#1 sales
perishable_class = A_B('perishable','class')
type_class = A_B('type','class')

#2 sales
city_class = A_B('city','class')

#3 sales
state_class = A_B('state','class')
state_cluster = A_B('state','cluster')

#4 sales
cluster_class = A_B('cluster','class')
cluster_store = A_B('cluster','store_nbr')

#5 sales
class_store = A_B('class','store_nbr')
country_item = country_B('item_nbr')

print("\nPreparing transactions..........................")
#0_trans
family_item_trans = A_B('family','item_nbr','transactions')
type_item_trans =  A_B('type','item_nbr','transactions')
cluster_item_trans =  A_B('cluster','item_nbr','transactions')

#1_trans
type_class_trans = A_B('type','class','transactions')
perishable_class_trans = A_B('perishable','class','transactions')
country_store_trans = country_B('store_nbr','transactions')

#2_trans
city_class_trans = A_B('city','class','transactions')
state_class_trans = A_B('state','class','transactions')
cluster_store_trans = A_B('cluster','store_nbr','transactions')
cluster_class_trans = A_B('cluster','class','transactions')
class_store_trans = A_B('class','store_nbr','transactions') 

Preparing unit_sales..........................
the shape of family_item_nbr is (4018, 227): 
the shape of country_store_nbr is (54, 227): 
the shape of country_perishable is (2, 227): 
the shape of country_class is (334, 227): 
the shape of cluster_item_nbr is (56638, 227): 
the shape of state_item_nbr is (52476, 227): 
the shape of type_item_nbr is (16741, 227): 
the shape of city_item_nbr is (68912, 227): 
the shape of perishable_class is (334, 227): 
the shape of type_class is (1544, 227): 
the shape of city_class is (6412, 227): 
the shape of state_class is (4765, 227): 
the shape of state_cluster is (37, 227): 
the shape of cluster_class is (5181, 227): 
the shape of cluster_store_nbr is (54, 227): 
the shape of class_store_nbr is (15826, 227): 
the shape of country_item_nbr is (4018, 227): 
Preparing transactions..........................
the shape of family_item_nbr is (4018, 227): 
the shape of type_item_nbr is (16741, 227): 
the shape of cluster_item_nbr is (56638, 227): 
the 

In [11]:
def get_timespan(df, dt, minus, periods, freq='D'):
    return df[pd.date_range(dt - timedelta(days=minus), periods=periods, freq=freq)]
def get_timespan_2(df, dt, minus, periods, freq='D'):
    return df[pd.date_range(dt + timedelta(days=minus), periods=periods, freq=freq)]

def get_nearwd(date,b_date):
    date_list = pd.date_range(date-timedelta(140),periods=21,freq='7D').date
    result = date_list[date_list<=b_date][-1]
    return result

In [12]:
def X_join_A_B_fea( X, A_B, t2017, str_A, str_B, ListA, ListB, str_type='sales'):## X
    X_A_B = pd.DataFrame({
    })
    for i in ListA:
        if(i=='mean3'): X_A_B["{}_{}_{}_mean3".format(str_A,str_B,str_type)]=get_timespan(
            A_B, t2017, 3, 3).mean(axis=1).values
        elif(i=='mean7'):X_A_B["{}_{}_{}_mean7".format(str_A,str_B,str_type)]=get_timespan(
            A_B, t2017, 7, 7).mean(axis=1).values
        elif(i=='mean14'):X_A_B["{}_{}_{}_mean14".format(str_A,str_B,str_type)]=get_timespan(
            A_B, t2017, 14, 14).mean(axis=1).values
        elif(i=='mean30'):X_A_B["{}_{}_{}_mean30".format(str_A,str_B,str_type)]=get_timespan(
            A_B, t2017, 30, 30).mean(axis=1).values
        elif(i=='mean60'):X_A_B["{}_{}_{}_mean60".format(str_A,str_B,str_type)]=get_timespan(
            A_B, t2017, 60, 60).mean(axis=1).values
        
        elif(i=='std3'):X_A_B["{}_{}_{}_std3".format(str_A,str_B,str_type)]=get_timespan(
            A_B, t2017, 3, 3).std(axis=1).values
        elif(i=='std7'):X_A_B["{}_{}_{}_std7".format(str_A,str_B,str_type)]=get_timespan(
            A_B, t2017, 7, 7).std(axis=1).values
        elif(i=='std14'):X_A_B["{}_{}_{}_std14".format(str_A,str_B,str_type)]=get_timespan(
            A_B, t2017, 14, 14).std(axis=1).values
        elif(i=='std30'):X_A_B["{}_{}_{}_std30".format(str_A,str_B,str_type)]=get_timespan(
            A_B, t2017, 30, 30).std(axis=1).values
        elif(i=='std60'):X_A_B["{}_{}_{}_std60".format(str_A,str_B,str_type)]=get_timespan(
            A_B, t2017, 60, 60).std(axis=1).values
            
        elif(i=='min3'):X_A_B["{}_{}_{}_min3".format(str_A,str_B,str_type)]=get_timespan(
            A_B, t2017, 3, 3).min(axis=1).values
        elif(i=='min7'):X_A_B["{}_{}_{}_min7".format(str_A,str_B,str_type)]=get_timespan(
            A_B, t2017, 7, 7).min(axis=1).values
        elif(i=='min14'):X_A_B["{}_{}_{}_min14".format(str_A,str_B,str_type)]=get_timespan(
            A_B, t2017, 14, 14).min(axis=1).values
        elif(i=='min30'):X_A_B["{}_{}_{}_min30".format(str_A,str_B,str_type)]=get_timespan(
            A_B, t2017, 30, 30).min(axis=1).values
        elif(i=='min60'):X_A_B["{}_{}_{}_min60".format(str_A,str_B,str_type)]=get_timespan(
            A_B, t2017, 60, 60).min(axis=1).values
            
        elif(i=='max3'):X_A_B["{}_{}_{}_max3".format(str_A,str_B,str_type)]=get_timespan(
            A_B, t2017, 3, 3).max(axis=1).values
        elif(i=='max7'):X_A_B["{}_{}_{}_max7".format(str_A,str_B,str_type)]=get_timespan(
            A_B, t2017, 7, 7).max(axis=1).values
        elif(i=='max14'):X_A_B["{}_{}_{}_max14".format(str_A,str_B,str_type)]=get_timespan(
            A_B, t2017, 14, 14).max(axis=1).values
        elif(i=='max30'):X_A_B["{}_{}_{}_max30".format(str_A,str_B,str_type)]=get_timespan(
            A_B, t2017, 30, 30).max(axis=1).values
        elif(i=='max60'):X_A_B["{}_{}_{}_max60".format(str_A,str_B,str_type)]=get_timespan(
            A_B, t2017, 60, 60).max(axis=1).values
            
        elif(i=='median3'):X_A_B["{}_{}_{}_median3".format(str_A,str_B,str_type)]=get_timespan(
            A_B, t2017, 3, 3).median(axis=1).values
        elif(i=='median7'):X_A_B["{}_{}_{}_median7".format(str_A,str_B,str_type)]=get_timespan(
            A_B, t2017, 7, 7).median(axis=1).values
        elif(i=='median14'):X_A_B["{}_{}_{}_median14".format(str_A,str_B,str_type)]=get_timespan(
            A_B, t2017, 14, 14).median(axis=1).values
        elif(i=='median30'):X_A_B["{}_{}_{}_median30".format(str_A,str_B,str_type)]=get_timespan(
            A_B, t2017, 30, 30).median(axis=1).values
        elif(i=='median60'):X_A_B["{}_{}_{}_median60".format(str_A,str_B,str_type)]=get_timespan(
            A_B, t2017, 60, 60).median(axis=1).values
    for i in ListB:                 ####如果第一轮需要加星期的特征，则这边需要修改
        if(i=='mean4'):
            for j in range(7):
                X_A_B['{}_{}_week{}_mean4'.format(str_A,str_B,j)] = get_timespan(A_B, t2017, 28-j, 4, 
                                                                     freq='7D').mean(axis=1).values  
        elif(i=='mean20'):
            for j in range(7):
                X_A_B['{}_{}_week{}_mean20'.format(str_A,str_B,j)] = get_timespan(A_B, t2017, 140-j, 20, 
                                                                      freq='7D').mean(axis=1).values
        elif(i=='median4'):
            for j in range(7):
                X_A_B['{}_{}_week{}_median4'.format(str_A,str_B,j)] = get_timespan(A_B, t2017, 28-j, 4, 
                                                                     freq='7D').median(axis=1).values
        elif(i=='median20'):
            for j in range(7):
                X_A_B['{}_{}_week{}_median20'.format(str_A,str_B,j)] = get_timespan(A_B, t2017, 140-j, 20, 
                                                                      freq='7D').median(axis=1).values
        elif(i=='min4'):
            for j in range(7):
                X_A_B['{}_{}_week{}_min4'.format(str_A,str_B,j)] = get_timespan(A_B, t2017, 28-j, 4, 
                                                                     freq='7D').min(axis=1).values
        elif(i=='min20'):
            for j in range(7):
                X_A_B['{}_{}_week{}_min20'.format(str_A,str_B,j)] = get_timespan(A_B, t2017, 140-j, 20, 
                                                                      freq='7D').min(axis=1).values
        elif(i=='max4'):
            for j in range(7):
                X_A_B['{}_{}_week{}_max4'.format(str_A,str_B,j)] = get_timespan(A_B, t2017, 28-j, 4, 
                                                                     freq='7D').max(axis=1).values
        elif(i=='max20'):
            for j in range(7):
                X_A_B['{}_{}_week{}_max20'.format(str_A,str_B,j)] = get_timespan(A_B, t2017, 140-j, 20, 
                                                                      freq='7D').max(axis=1).values
        elif(i=='std4'):
            for j in range(7):
                X_A_B['{}_{}_week{}_std4'.format(str_A,str_B,j)] = get_timespan(A_B, t2017, 28-j, 4, 
                                                                     freq='7D').std(axis=1).values
        elif(i=='std20'):
            for j in range(7):
                X_A_B['{}_{}_week{}_std20'.format(str_A,str_B,j)] = get_timespan(A_B, t2017, 140-j, 20, 
                                                                      freq='7D').std(axis=1).values
    
    if (str_A=='country'):
        X_A_B["{}".format(str_B)] = A_B.reset_index()[str_B]
        X = X.join( X_A_B.set_index([str_B]), on=[str_B], how='left')
    else :
        X_A_B["{}".format(str_A)] = A_B.reset_index()[str_A]
        X_A_B["{}".format(str_B)] = A_B.reset_index()[str_B]
        X = X.join( X_A_B.set_index([str_A,str_B]), on=[str_A,str_B], how='left')
        
    return X

# Predict future 6 days

In [15]:
def prepare_dataset(t2017, is_train=True):
    X = pd.DataFrame({
        "day_1_2017": get_timespan(df_2017, t2017, 1, 1).values.ravel(),
        "promo_14_2017": get_timespan(promo_2017, t2017, 14, 14).sum(axis=1).values,
        "promo_60_2017": get_timespan(promo_2017, t2017, 60, 60).sum(axis=1).values,
        "promo_140_2017": get_timespan(promo_2017, t2017, 140, 140).sum(axis=1).values,
        "std_140_past": get_timespan(df_2017, t2017, 140, 140).std(axis=1).values,#11th
        "store_nbr":column_names['store_nbr'],#13
        "city":column_names['city'],
        "country":column_names['country'],
        "item_nbr":column_names['item_nbr'],
        "family":column_names['family'],
        "class":column_names['class'],#13
        "unpromo_16aft_2017":(1-get_timespan(promo_2017, t2017+timedelta(16), 16, 16)).sum(axis=1).values,
        
        "NaN_in_pro_7": get_timespan(promo_2017_train_NaN, t2017, 7, 7).isnull().sum(axis=1).values,
        "NaN_in_pro_30": get_timespan(promo_2017_train_NaN, t2017, 30, 30).isnull().sum(axis=1).values,
        "NaN_in_pro_14": get_timespan(promo_2017_train_NaN, t2017, 14, 14).isnull().sum(axis=1).values,
        "NaN_in_sales_7": get_timespan(df_2017_NaN, t2017, 7, 7).isnull().sum(axis=1).values,
        "NaN_in_sales_14": get_timespan(df_2017_NaN, t2017, 14, 14).isnull().sum(axis=1).values,
        "NaN_in_sales_30": get_timespan(df_2017_NaN, t2017, 30, 30).isnull().sum(axis=1).values,
    })
    
    X = X_join_A_B_fea(X,country_item,t2017,'country','item_nbr',['mean3','mean7','mean14','mean30','mean60'],[])
    X = X_join_A_B_fea(X,city_item,t2017,'city','item_nbr',['std7','mean3','mean7','mean14','mean30','mean60'],[])
    
    X.drop(['city','country','family'],axis=1,inplace=True)  #13th
    
    for i in range(14):
        X["weekBef_{}_mean_3".format(15-i)] = get_timespan_2(
            df_2017, t2017-timedelta(16), i, 3).mean(axis=1).values
    
    for i in range(16):
        X["promo_{}".format(i)] = promo_2017[
            t2017 + timedelta(days=i)].values.astype(np.uint8)
        for j in [14]:
            X["aft_promo_{}{}".format(i,j)] = (promo_2017[
                t2017 + timedelta(days=i)]-1).values.astype(np.uint8)
            X["aft_promo_{}{}".format(i,j)] = X["aft_promo_{}{}".format(i,j)]\
                                        *X['promo_{}_2017'.format(j)]
        if i ==15:
            X["bf_unpromo_{}".format(i)]=0
        else:
            X["bf_unpromo_{}".format(i)] = (1-get_timespan(
                    promo_2017, t2017+timedelta(16), 16-i, 16-i)).iloc[:,1:].sum(
                            axis=1).values / (15-i) * X['promo_{}'.format(i)]

    for i in range(7):
        X['mean_4_dow{}_2017'.format(i)] = get_timespan(df_2017, t2017, 28-i, 4, freq='7D').mean(axis=1).values
        #X['mean_12_dow{}_2017'.format(i)] = get_timespan(df_2017, t2017, 84-i, 12, freq='7D').mean(axis=1).values
        X['mean_20_dow{}_2017'.format(i)] = get_timespan(df_2017, t2017, 140-i, 20, freq='7D').mean(axis=1).values        
        
        date = get_nearwd(t2017+timedelta(i),t2017)
        ahead = (t2017-date).days
        if ahead!=0:
            X['ahead0_{}'.format(i)] = get_timespan(df_2017, date+timedelta(ahead), ahead, ahead).mean(axis=1).values
            X['ahead7_{}'.format(i)] = get_timespan(df_2017, date+timedelta(ahead), ahead+7, ahead+7).mean(axis=1).values
        if((i==0) or (i==6)):
            for m in [3,7,14,30,60,140]:
                X["mean_{}_2017_{}1".format(m,i)]= get_timespan(df_2017, date,m, m).\
                    mean(axis=1).values
    return X
def prepare_dataset_y(t2017):
    y = df_2017[
        pd.date_range(t2017, periods=6) ].values
    return y


print("Preparing dataset...")

t2017 = date(2017, 6, 7)
X_l, y_l = [], []
for i in range(10):
    delta = timedelta(days=7 * i)
    X_tmp = prepare_dataset(
        t2017 + delta
    )
    y_tmp = prepare_dataset_y(
        t2017 + delta
    )
    X_l.append(X_tmp)
    y_l.append(y_tmp)
X_train = pd.concat(X_l, axis=0)
y_train = np.concatenate(y_l, axis=0)
del X_l, y_l
X_test = prepare_dataset(date(2017, 8, 16))

#X_train.to_csv('X_train_6.csv', float_format='%.6f', index=None)
#pd.DataFrame(y_train).to_csv('y_train_6.csv', float_format='%.6f', index=None)
#X_val.to_csv('X_val_6.csv', float_format='%.6f', index=None)
#pd.DataFrame(y_val).to_csv('y_val_6.csv', float_format='%.6f', index=None)
#X_test.to_csv('X_test.csv', float_format='%.6f', index=None)
#pd.DataFrame(index=df_2017.index).to_csv('stores_items.csv')
#df_test[['id']].to_csv('test_ids.csv')###非常重要，免得下次再跑了
                                      ###非常重要，免得下次再跑了
                                      ###非常重要，免得下次再跑了

Preparing dataset...


In [16]:
print("Training and predicting models...")
params = {
    'num_leaves': 31,
    'objective': 'regression',
    'min_data_in_leaf': 300,
    'learning_rate': 0.1,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 2,
    'metric': 'l2_root',
    'num_threads': 32
}

MAX_ROUNDS = 500
test_pred = []
cate_vars = []
for i in range(6):
    print("=" * 50)
    print("Step %d" % (i+1))
    print("=" * 50)
    dtrain = lgb.Dataset(
        X_train, label=y_train[:, i],
        categorical_feature=cate_vars,
        weight=pd.concat([items["perishable"]] * 10) * 0.25 + 1
    )
    bst = lgb.train(
        params, dtrain, num_boost_round=MAX_ROUNDS,
        valid_sets=[dtrain], verbose_eval=50
    )
    print("\n".join(("%s: %.2f" % x) for x in sorted(
        zip(X_train.columns, bst.feature_importance("gain")),
        key=lambda x: x[1], reverse=True
    )))
    test_pred.append(bst.predict(
        X_test, num_iteration=MAX_ROUNDS))

Training and predicting models...
Step 1


E:\DQS\ANACONDA\SETUP\lib\site-packages\lightgbm\basic.py:1005: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


[50]	training's rmse: 0.546886
[100]	training's rmse: 0.541653
[150]	training's rmse: 0.539197
[200]	training's rmse: 0.537502
[250]	training's rmse: 0.536175
[300]	training's rmse: 0.535012
[350]	training's rmse: 0.534004
[400]	training's rmse: 0.53304
[450]	training's rmse: 0.532134
[500]	training's rmse: 0.531329
mean_7_2017_01: 2444783.99
ahead7_5: 1213369.63
mean_14_2017_01: 923434.82
ahead0_2: 269043.93
mean_30_2017_01: 223707.31
ahead7_6: 184735.26
ahead0_5: 161776.26
mean_20_dow0_2017: 123293.71
day_1_2017: 102032.94
promo_0: 98476.37
mean_4_dow0_2017: 87766.42
bf_unpromo_0: 60397.28
aft_promo_014: 33402.56
bf_unpromo_7: 24857.98
mean_60_2017_01: 21395.12
std_140_past: 18715.72
weekBef_13_mean_3: 18553.59
store_nbr: 14645.09
promo_14_2017: 13457.59
country_item_nbr_sales_mean3: 11769.42
bf_unpromo_14: 11735.45
mean_3_2017_01: 11032.43
weekBef_2_mean_3: 10415.43
class: 9273.80
ahead0_3: 8521.81
country_item_nbr_sales_mean60: 8188.25
mean_60_2017_61: 8026.97
ahead0_6: 5547.44
NaN

Step 4
[50]	training's rmse: 0.588441
[100]	training's rmse: 0.582115
[150]	training's rmse: 0.578807
[200]	training's rmse: 0.576538
[250]	training's rmse: 0.574833
[300]	training's rmse: 0.573394
[350]	training's rmse: 0.572071
[400]	training's rmse: 0.570865
[450]	training's rmse: 0.569789
[500]	training's rmse: 0.568767
ahead7_2: 1725762.98
ahead7_1: 1584055.97
ahead0_2: 1028301.17
ahead0_1: 630159.73
mean_30_2017_01: 391732.23
mean_4_dow3_2017: 372226.13
mean_20_dow3_2017: 213634.95
mean_14_2017_01: 159377.77
mean_60_2017_01: 98875.86
promo_3: 89635.03
weekBef_4_mean_3: 54981.67
ahead0_3: 46050.33
aft_promo_314: 29919.01
unpromo_16aft_2017: 25251.06
bf_unpromo_3: 20472.06
mean_60_2017_61: 20259.33
country_item_nbr_sales_mean3: 18200.80
mean_30_2017_61: 17466.18
std_140_past: 16821.06
mean_4_dow4_2017: 13303.67
country_item_nbr_sales_mean60: 13094.25
store_nbr: 12416.76
class: 12367.41
promo_14_2017: 11795.70
item_nbr: 11051.88
mean_4_dow2_2017: 7899.32
mean_7_2017_01: 7719.30
prom

# Predict middle 7 days

In [17]:
def prepare_dataset_y(t2017):
    y = df_2017[
        pd.date_range(t2017, periods=13) ].values
    return y


print("Preparing dataset...")

t2017 = date(2017, 6, 7)
X_l, y_l = [], []
for i in range(9):
    delta = timedelta(days=7 * i)
    X_tmp = prepare_dataset(
        t2017 + delta
    )
    y_tmp = prepare_dataset_y(
        t2017 + delta
    )
    X_l.append(X_tmp)
    y_l.append(y_tmp)
X_train = pd.concat(X_l, axis=0)
y_train = np.concatenate(y_l, axis=0)
del X_l, y_l

#X_train.to_csv('X_train_7.csv', float_format='%.6f', index=None)
#pd.DataFrame(y_train).to_csv('y_train_7.csv', float_format='%.6f', index=None)
#X_val.to_csv('X_val_7.csv', float_format='%.6f', index=None)
#pd.DataFrame(y_val_2).to_csv('y_val_7.csv', float_format='%.6f', index=None)
#X_test.to_csv('X_test.csv', float_format='%.6f', index=None)
#pd.DataFrame(index=df_2017.index).to_csv('stores_items.csv')
#df_test[['id']].to_csv('test_ids.csv')###非常重要，免得下次再跑了
                                      ###非常重要，免得下次再跑了
                                      ###非常重要，免得下次再跑了

Preparing dataset...


In [18]:
print("Training and predicting models...")
for i in range(6,13):
    print("=" * 50)
    print("Step %d" % (i+1))
    print("=" * 50)
    dtrain = lgb.Dataset(
        X_train, label=y_train[:, i],
        categorical_feature=cate_vars,
        weight=pd.concat([items["perishable"]] * 9) * 0.25 + 1
    )
    bst = lgb.train(
        params, dtrain, num_boost_round=MAX_ROUNDS,
        valid_sets=[dtrain], verbose_eval=50
    )
    print("\n".join(("%s: %.2f" % x) for x in sorted(
        zip(X_train.columns, bst.feature_importance("gain")),
        key=lambda x: x[1], reverse=True
    )))
    test_pred.append(bst.predict(
        X_test, num_iteration=MAX_ROUNDS))

Training and predicting models...
Step 7


E:\DQS\ANACONDA\SETUP\lib\site-packages\lightgbm\basic.py:1005: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


[50]	training's rmse: 0.59029
[100]	training's rmse: 0.584327
[150]	training's rmse: 0.581597
[200]	training's rmse: 0.579293
[250]	training's rmse: 0.577702
[300]	training's rmse: 0.576232
[350]	training's rmse: 0.57497
[400]	training's rmse: 0.573679
[450]	training's rmse: 0.572447
[500]	training's rmse: 0.571282
ahead7_5: 1479938.11
mean_30_2017_01: 1267027.07
ahead7_1: 576212.86
ahead7_4: 421718.13
ahead7_6: 206073.33
mean_20_dow6_2017: 187757.63
mean_4_dow6_2017: 122226.36
promo_6: 116150.61
weekBef_2_mean_3: 86350.60
mean_14_2017_01: 68430.40
ahead7_2: 63023.90
ahead0_5: 56836.28
mean_60_2017_01: 53793.24
bf_unpromo_6: 51391.00
ahead0_3: 51128.23
aft_promo_614: 25807.44
country_item_nbr_sales_mean3: 19429.16
unpromo_16aft_2017: 19216.17
std_140_past: 18432.63
class: 15320.51
item_nbr: 14468.78
mean_3_2017_01: 12575.39
promo_14_2017: 12078.35
country_item_nbr_sales_mean60: 11607.96
day_1_2017: 10518.25
ahead0_1: 9986.80
mean_30_2017_61: 9981.32
mean_60_2017_61: 8908.60
mean_20_dow

Step 10
[50]	training's rmse: 0.592489
[100]	training's rmse: 0.585339
[150]	training's rmse: 0.581706
[200]	training's rmse: 0.578886
[250]	training's rmse: 0.576794
[300]	training's rmse: 0.574989
[350]	training's rmse: 0.573491
[400]	training's rmse: 0.572142
[450]	training's rmse: 0.570839
[500]	training's rmse: 0.569664
mean_30_2017_01: 1592816.66
mean_14_2017_01: 692251.92
ahead7_1: 630943.90
mean_7_2017_01: 621918.10
mean_20_dow2_2017: 612677.05
mean_4_dow2_2017: 383914.16
promo_9: 181488.53
ahead0_1: 138132.98
std_140_past: 39751.36
mean_60_2017_01: 35609.29
weekBef_5_mean_3: 31767.67
class: 24956.88
aft_promo_914: 22269.07
unpromo_16aft_2017: 21137.73
mean_60_2017_61: 18883.63
item_nbr: 18399.37
country_item_nbr_sales_mean3: 16391.61
bf_unpromo_9: 16103.55
ahead7_6: 15612.89
bf_unpromo_2: 14525.13
country_item_nbr_sales_mean60: 12402.08
store_nbr: 12222.71
promo_140_2017: 11669.00
promo_14_2017: 11249.76
ahead0_2: 10802.33
day_1_2017: 9179.24
country_item_nbr_sales_mean30: 878

Step 13
[50]	training's rmse: 0.608027
[100]	training's rmse: 0.601131
[150]	training's rmse: 0.59767
[200]	training's rmse: 0.595176
[250]	training's rmse: 0.593246
[300]	training's rmse: 0.591506
[350]	training's rmse: 0.589991
[400]	training's rmse: 0.588569
[450]	training's rmse: 0.587401
[500]	training's rmse: 0.586091
mean_30_2017_01: 2086860.11
ahead7_4: 805219.00
ahead7_3: 432989.17
mean_14_2017_01: 322326.61
mean_60_2017_01: 319477.72
ahead7_5: 176501.62
mean_20_dow5_2017: 149951.22
ahead0_3: 123169.80
promo_12: 115792.30
weekBef_2_mean_3: 87569.07
mean_4_dow5_2017: 64440.65
mean_60_2017_61: 57895.01
mean_3_2017_01: 54082.16
ahead0_5: 37198.99
country_item_nbr_sales_mean3: 27820.60
unpromo_16aft_2017: 24543.39
mean_30_2017_61: 23953.92
aft_promo_1214: 23158.18
item_nbr: 22774.95
bf_unpromo_12: 21034.18
std_140_past: 20590.46
class: 20334.51
country_item_nbr_sales_mean60: 17948.81
mean_140_2017_01: 17498.75
promo_13: 13946.98
ahead7_1: 10838.97
weekBef_15_mean_3: 10134.64
count

# Predict last 3 days

In [19]:
def prepare_dataset_y(t2017):
    y = df_2017[
        pd.date_range(t2017, periods=16) ].values
    return y


print("Preparing dataset...")

t2017 = date(2017, 6, 7)
X_l, y_l = [], []
for i in range(8):
    delta = timedelta(days=7 * i)
    X_tmp = prepare_dataset(
        t2017 + delta
    )
    y_tmp = prepare_dataset_y(
        t2017 + delta
    )
    X_l.append(X_tmp)
    y_l.append(y_tmp)
X_train = pd.concat(X_l, axis=0)
y_train = np.concatenate(y_l, axis=0)
del X_l, y_l


#X_train.to_csv('X_train_3.csv', float_format='%.6f', index=None)
#pd.DataFrame(y_train).to_csv('y_train_3.csv', float_format='%.6f', index=None)
#X_val.to_csv('X_val_3.csv', float_format='%.6f', index=None)
#pd.DataFrame(y_val_3).to_csv('y_val_3.csv', float_format='%.6f', index=None)
#X_test.to_csv('X_test.csv', float_format='%.6f', index=None)
#pd.DataFrame(index=df_2017.index).to_csv('stores_items.csv')
#df_test[['id']].to_csv('test_ids.csv')###非常重要，免得下次再跑了
                                      ###非常重要，免得下次再跑了
                                      ###非常重要，免得下次再跑了

Preparing dataset...


In [20]:
print("Training and predicting models...")
for i in range(13,16):
    print("=" * 50)
    print("Step %d" % (i+1))
    print("=" * 50)
    dtrain = lgb.Dataset(
        X_train, label=y_train[:, i],
        categorical_feature=cate_vars,
        weight=pd.concat([items["perishable"]] * 8) * 0.25 + 1
    )
    bst = lgb.train(
        params, dtrain, num_boost_round=MAX_ROUNDS,
        valid_sets=[dtrain], verbose_eval=50
    )
    print("\n".join(("%s: %.2f" % x) for x in sorted(
        zip(X_train.columns, bst.feature_importance("gain")),
        key=lambda x: x[1], reverse=True
    )))
    test_pred.append(bst.predict(
        X_test, num_iteration=MAX_ROUNDS))

Training and predicting models...
Step 14


E:\DQS\ANACONDA\SETUP\lib\site-packages\lightgbm\basic.py:1005: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


[50]	training's rmse: 0.605266
[100]	training's rmse: 0.597917
[150]	training's rmse: 0.594368
[200]	training's rmse: 0.591763
[250]	training's rmse: 0.589808
[300]	training's rmse: 0.588006
[350]	training's rmse: 0.586393
[400]	training's rmse: 0.584883
[450]	training's rmse: 0.583465
[500]	training's rmse: 0.582132
mean_30_2017_01: 2160710.30
mean_14_2017_01: 313181.83
ahead7_4: 285784.66
ahead7_5: 274272.54
mean_20_dow6_2017: 271505.62
mean_60_2017_01: 213635.29
ahead7_6: 211433.45
promo_13: 204734.24
ahead0_1: 67802.43
mean_4_dow6_2017: 58644.10
weekBef_2_mean_3: 49277.99
ahead0_5: 45389.86
item_nbr: 25060.63
class: 22954.49
std_140_past: 22522.00
country_item_nbr_sales_mean3: 21002.56
weekBef_15_mean_3: 17910.29
unpromo_16aft_2017: 15214.99
day_1_2017: 14206.59
country_item_nbr_sales_mean60: 14135.63
aft_promo_1314: 9827.41
promo_14_2017: 9742.45
country_item_nbr_sales_mean30: 9585.23
mean_3_2017_01: 9542.36
mean_20_dow5_2017: 9292.32
mean_20_dow1_2017: 9102.44
mean_140_2017_01: 7

In [21]:
print("Making submission...")
y_test = np.array(test_pred).transpose()
df_preds = pd.DataFrame(
    y_test, index=df_2017.index,
    columns=pd.date_range("2017-08-16", periods=16)
).stack().to_frame("unit_sales")
df_preds.index.set_names(["store_nbr", "item_nbr", "date"], inplace=True)

submission = df_test[["id"]].join(df_preds, how="left").fillna(0).reset_index()
submission["unit_sales"] = np.clip(np.expm1(submission["unit_sales"]), 0, 10000)
submission.loc[~submission.item_nbr.isin(item_nbr_u),'unit_sales']=0
submission[['id','unit_sales']].to_csv('E:/DQS/kaggle/LGBM_20th.csv', float_format='%.4f', index=None)

Making submission...
